## Foolbox Example Implementation W/Tensorflow

In [1]:
# Tensorflow logging: OFF
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
# Imports
import tensorflow as tf
import eagerpy as ep
from foolbox import TensorFlowModel, accuracy, samples, Model
from foolbox.attacks import LinfPGD

print(f'Tensorflow Version: {tf.__version__}')

Tensorflow Version: 2.15.0


In [3]:
# Check GPU availability and use it... if you have one
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print("GPU Availability:", len(gpus), "Physical GPUs &", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
else:
    print("No GPUs detected.")

GPU Availability: 1 Physical GPUs & 1 Logical GPUs


---

In [4]:
model = tf.keras.applications.ResNet50(weights="imagenet")

In [5]:
pre = dict(flip_axis=-1, mean=[104.0, 116.0, 123.0])

In [6]:
fmodel: Model = TensorFlowModel(model, bounds=(0, 255), preprocessing=pre)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


In [7]:
fmodel = fmodel.transform_bounds((0,1))

In [8]:
images, labels = ep.astensors(*samples(fmodel, dataset="imagenet", batchsize=1))

In [9]:
clean_acc = accuracy(fmodel, images, labels)

In [10]:
print(f"clean accuracy:  {clean_acc * 100:.1f} %")

clean accuracy:  100.0 %


In [11]:
attack = LinfPGD()

In [12]:
epsilons = [
    0.0,
    0.0002,
    0.0005,
    0.0008,
    0.001,
    0.0015,
    0.002,
    0.003,
    0.01,
    0.1,
    0.3,
    0.5,
    1.0,
]
raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=epsilons)

In [13]:
robust_accuracy = 1 - success.float32().mean(axis=-1)

In [15]:
print("robust accuracy for perturbations with")
for eps, acc in zip(epsilons, robust_accuracy):
    print(f"Linf norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")

robust accuracy for perturbations with
Linf norm ≤ 0.0   : 100.0 %
Linf norm ≤ 0.0002: 100.0 %
Linf norm ≤ 0.0005: 100.0 %
Linf norm ≤ 0.0008: 100.0 %
Linf norm ≤ 0.001 : 100.0 %
Linf norm ≤ 0.0015:  0.0 %
Linf norm ≤ 0.002 :  0.0 %
Linf norm ≤ 0.003 :  0.0 %
Linf norm ≤ 0.01  :  0.0 %
Linf norm ≤ 0.1   :  0.0 %
Linf norm ≤ 0.3   :  0.0 %
Linf norm ≤ 0.5   :  0.0 %
Linf norm ≤ 1.0   :  0.0 %


In [16]:
print("robust accuracy for perturbations with")
for eps, advs_ in zip(epsilons, clipped_advs):
    acc2 = accuracy(fmodel, advs_, labels)
    print(f"  Linf norm ≤ {eps:<6}: {acc2 * 100:4.1f} %")
    print("    perturbation sizes:")
    perturbation_sizes = (advs_ - images).norms.linf(axis=(1, 2, 3)).numpy()
    print("    ", str(perturbation_sizes).replace("\n", "\n" + "    "))
    if acc2 == 0:
        break

robust accuracy for perturbations with
  Linf norm ≤ 0.0   : 100.0 %
    perturbation sizes:
     [0.]
  Linf norm ≤ 0.0002: 100.0 %
    perturbation sizes:
     [0.0002]
  Linf norm ≤ 0.0005: 100.0 %
    perturbation sizes:
     [0.00050002]
  Linf norm ≤ 0.0008: 100.0 %
    perturbation sizes:
     [0.00080001]
  Linf norm ≤ 0.001 : 100.0 %
    perturbation sizes:
     [0.001]
  Linf norm ≤ 0.0015:  0.0 %
    perturbation sizes:
     [0.00150001]
